# Aula 1 - Parte Prática - Agentes & Ambientes

## Introdução

Nesse primeiro notebook iremos aprender sobre a API do [OpenAI Gym](http://gym.openai.com/) e começaremos a implementar alguns componentes de um agente de RL.
Ao final dessa parte prática teremos implementado o **ciclo de interação Agente-Ambiente** que nos permitirá futuramente treinar o agente e avaliar sua performance.

### Objetivos:

- Relacionar os conceitos de Processos de Decisão Markovianos (MDPs) com os atributos e métodos de um ambiente definido com o OpenAI Gym;
- Familiarizar-se com os componentes básicos de um agente de RL;
- Implementar o ciclo de interação Agente-Ambiente; e
- Implementar um primeiro agente aleatório e avaliar sua performance

## Imports

> **Atenção:** não se esqueça de executar todos os `imports` necessários antes prosseguir com o tutorial. 

In [2]:
import gym

## 1. Ambientes no OpenAI Gym

Um ambiente no OpenAI Gym encapsula um simulador com o qual um agente pode interagir. Nesse simulador, a cada instante $t$ o agente deve escolhar uma ação $\mathbf{a}_t \in \mathcal{A}$ a ser executada. Ao receber essa ação, o ambiente tem seu estado $\mathbf{s}_t \in \mathcal{S}$ alterado para outro estado $\mathbf{s}_{t+1} \in \mathcal{S}$ e devolve para o agente uma observação (que pode ou não corresponder ao estado) e uma recompensa/punição $r_t \in \mathbb{R}$.

O pacote Gym conta com inúmeros [ambientes pré-programados](http://gym.openai.com/envs/) e prontos para serem usados para testar algoritmos de RL. Nessa parte prática, começaremos a explorar alguns ambientes mais simples a fim de nos familiarizarmos com os principais conceitos de modelagem da biblioteca Gym.

Para carregar um ambiente disponível no Gym basta chamar a função `gym.make` passando como argumento o identificador do ambiente:

In [14]:
env = gym.make("MountainCar-v0")

> **Atenção**: antes de usar um ambiente do Gym com o qual você não está familiarizado pode ser útil ler a sua documentação online.
> Para o `MountainCar-v0` acesse o link [http://gym.openai.com/envs/MountainCar-v0/](http://gym.openai.com/envs/MountainCar-v0/).

### 1.1. Espaço de estados e ações

Todo agente de RL deve conhecer quais ações pode tomar no ambiente e também quais as características das variáveis de observações que aquele ambiente lhe disponibiliza. Para acessar o espaço de estados e ações, um ambiente do gym disponibiliza os atributos `env.observation_space` e `env.action_space`, respectivamente.

Note que essas informações serão importantes futuramente na definição das entradas e saídas das redes neurais artificiais que utilizaremos para representar a política $\pi_\theta$ do agente e também na criação de outros modelos.   

Todo `observation_space` tem associado seu tipo numérico (e.g., int, float,...) e as dimensões de uma observação. Além disso, é possível saber se o valor das variáveis observação são limitadas ou não, e se forem limitadas qual o valor mínimo e máximo.

In [45]:
obs_space = env.observation_space

print(obs_space.dtype)
print(obs_space.shape)
print(obs_space.bounded_above, obs_space.low)
print(obs_space.bounded_below, obs_space.high)

float32
(2,)
[ True  True] [-1.2  -0.07]
[ True  True] [0.6  0.07]


Para o `MountainCar-v0` observe que uma observação correponde a um vetor de números reais de tamanho 2 (i.e., um par de valores em ponto flutuante). Note que a primeira componente do vetor é limitada entre -1.2 e 0.6, e a segunda componente entre -0.07 e 0.07. 

Analogamente, todo `action_space` tem associado seu tipo numérico e as dimensões de uma ação.

In [52]:
action_space = env.action_space

print(action_space.dtype)
print(action_space.shape)
print(action_space.n)

int64
()
3


Note que para o `MountainCar-v0` o agente deverá escolher ações discretas (i.e., representadas por números inteiros). Observe também que o `shape==()` indica que uma ação é dada por um único escalar (e não um vetor como no caso do `observation_space`. Para acessar o número de possíveis valores da ação basta acessar o atributo `action_space.n`.

## 2. Agentes 